Count up sweeps from standing variation (origin pre-optimum, fixation post optimum) vs. from new mutations (both post-optimum)

In [1]:
#This script requires that reduce.sh has been executed from the BoGPoster directory
#The purpose of this notebook is to execute taht big loop below and picke the output...
#%matplotlib inline
#import matplotlib
#import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import feather  ##To get data to R
import math
import glob
import pickle

In [2]:
ff=glob.glob('../*.fixations.h5')

In [3]:
DATA=[]
for i in ff:
    fn=i
    i=i.strip('../')
    i=i.strip('.fixations.h5')
    i=i.split('_')
    PARAMS=dict(zip(i[0::2],i[1::2]))
    print PARAMS
    #break
    data=pd.read_hdf(fn)
    data.reset_index(inplace=True,drop=True)
    
    dg=data.groupby(['rep'])
    for i in dg.groups:
        dgi=dg.get_group(i)
        g2=dgi.groupby(['pos','esize','origin'])
        standing=0
        new=0
        for j in g2.groups:
            fi = g2.get_group(j)
            o=fi.iloc[0]['origin']
            f=fi.iloc[len(fi.index)-1]['generation']
            e=fi.iloc[0]['esize']
            rep=fi.iloc[0]['rep']
            if o < 10000 and f >= 10000:
                standing += 1
                fo=fi[fi.generation==10000].iloc[0]['freq']
                DATA.append({
                        'esize':e,
                        'origin':o,
                        'flen':(f-o+1),
                        'ftime':f,
                        'rep':rep,
                        'f0':fo,
                        'type':0,
                        'opt':float(PARAMS['OPT'])
                    })
            elif o > 10000 and f > 10000:
                DATA.append({
                        'esize':e,
                        'origin':o,
                        'flen':(f-o+1),
                        'ftime':f,
                        'rep':rep,
                        'f0':-1.0,
                        'type':1,
                        'opt':float(PARAMS['OPT'])
                    })
                    


{'H2': '0.2', 'OPT': '1', 'mu': '0.001'}
{'H2': '0.2', 'OPT': '0.5', 'mu': '0.001'}


Turn it all into a pd.DataFrame and then a feather file for processing in R and/or Python/Pandas:

In [4]:
df=pd.DataFrame(DATA)

In [5]:
df.head()

,esize,f0,flen,ftime,opt,origin,rep,type
0,-0.044386,-1.0000,3591.0,17786.0,1.0,14196.0,0.0,1
1,0.002162,-1.0000,3829.0,16554.0,1.0,12726.0,0.0,1
2,0.233656,-1.0000,5647.0,15649.0,1.0,10003.0,0.0,1
3,0.379240,0.0005,149.0,10147.0,1.0,9999.0,0.0,0
4,-0.027446,-1.0000,2211.0,16366.0,1.0,14156.0,0.0,1


In [6]:
feather.write_dataframe(df,'standing_vs_new.feather')

In [10]:
df[df.rep==0]

,esize,f0,flen,ftime,opt,origin,rep,type
0,-0.044386,-1.0000,3591.0,17786.0,1.0,14196.0,0.0,1
1,0.002162,-1.0000,3829.0,16554.0,1.0,12726.0,0.0,1
2,0.233656,-1.0000,5647.0,15649.0,1.0,10003.0,0.0,1
3,0.379240,0.0005,149.0,10147.0,1.0,9999.0,0.0,0
4,-0.027446,-1.0000,2211.0,16366.0,1.0,14156.0,0.0,1
5,-0.007042,-1.0000,3470.0,15570.0,1.0,12101.0,0.0,1
3164,-0.012076,-1.0000,2857.0,19687.0,0.5,16831.0,0.0,1
3165,0.055402,0.0710,3494.0,13259.0,0.5,9766.0,0.0,0
3166,0.124055,0.1075,757.0,10249.0,0.5,9493.0,0.0,0
3167,0.093769,-1.0000,989.0,11057.0,0.5,10069.0,0.0,1


In [7]:
g=df.groupby(['rep','opt','type'])

In [27]:
#Lesson: at first, this was not as easy as dplyr, but 
#I got it wrong in dplyr the first time and would not have noticed.
g['type'].value_counts().unstack().reset_index().groupby(['opt','type']).mean()

type             rep         0         1
opt type                                
0.5 0     510.600529  1.382275       NaN
    1     513.239814       NaN  2.089639
1.0 0     508.120160  1.616822       NaN
    1     508.973363       NaN  2.167592

In [28]:
g['type'].value_counts().unstack().reset_index().groupby(['opt','type']).min()

type      rep    0    1
opt type               
0.5 0     0.0  1.0  NaN
    1     0.0  NaN  1.0
1.0 0     0.0  1.0  NaN
    1     0.0  NaN  1.0

In [29]:
g['type'].value_counts().unstack().reset_index().groupby(['opt','type']).max()

type         rep    0    1
opt type                  
0.5 0     1023.0  4.0  NaN
    1     1023.0  NaN  7.0
1.0 0     1023.0  5.0  NaN
    1     1023.0  NaN  7.0

In [32]:
df.groupby(['opt','type']).mean()

esize        f0         flen         ftime        origin  \
opt type                                                                
0.5 0     0.158472  0.052603  1345.858373  11190.989474   9846.131100   
    1     0.044351 -1.000000  2437.890251  14469.967688  12033.077437   
1.0 0     0.213010  0.041191  1130.093311  10994.000000   9864.906689   
    1     0.109691 -1.000000  2200.355351  13942.031234  11742.675883   

                 rep  
opt type              
0.5 0     514.592344  
    1     517.397214  
1.0 0     501.423617  
    1     515.651818